   
# Advanced Video Analytics

The tutorial shows some techniques for developing advanced video analytics applications.

### Setup the environment variables, download model files and import dependencies

In [1]:
from IPython.display import HTML
import os
import time
import sys                                     
from pathlib import Path
sys.path.insert(0, str(Path().resolve().parent.parent.parent))
from demoTools.demoutils import *

In [2]:
!/opt/intel/openvino/bin/setupvars.sh

[setupvars.sh] OpenVINO environment initialized


In [3]:
!/opt/intel/openvino/deployment_tools/tools/model_downloader/downloader.py --name person-detection-retail-0013 -o models
!/opt/intel/openvino/deployment_tools/tools/model_downloader/downloader.py --name face-detection-adas-0001 -o models
!/opt/intel/openvino/deployment_tools/tools/model_downloader/downloader.py --name face-detection-adas-0001 -o models
!/opt/intel/openvino/deployment_tools/tools/model_downloader/downloader.py --name head-pose-estimation-adas-0001 -o models
!/opt/intel/openvino/deployment_tools/tools/model_downloader/downloader.py --name age-gender-recognition-retail-0013  -o models
!/opt/intel/openvino/deployment_tools/tools/model_downloader/downloader.py --name emotions-recognition-retail-0003  -o models
!/opt/intel/openvino/deployment_tools/tools/model_downloader/downloader.py --name facial-landmarks-35-adas-0002  -o models
!/opt/intel/openvino/deployment_tools/tools/model_downloader/downloader.py --name mobilenet-ssd -o models
!/opt/intel/openvino/deployment_tools/tools/model_downloader/downloader.py --name vehicle-license-plate-detection-barrier-0106 -o models
!/opt/intel/openvino/deployment_tools/tools/model_downloader/downloader.py --name vehicle-attributes-recognition-barrier-0039 -o models
!/opt/intel/openvino/deployment_tools/tools/model_downloader/downloader.py --name license-plate-recognition-barrier-0001 -o models

################|| Downloading models ||################

========== Downloading models/intel/person-detection-retail-0013/FP32/person-detection-retail-0013.xml
... 100%, 152 KB, 3406 KB/s, 0 seconds passed

========== Downloading models/intel/person-detection-retail-0013/FP32/person-detection-retail-0013.bin
... 100%, 2823 KB, 29521 KB/s, 0 seconds passed

========== Downloading models/intel/person-detection-retail-0013/FP16/person-detection-retail-0013.xml
... 100%, 152 KB, 21643 KB/s, 0 seconds passed

========== Downloading models/intel/person-detection-retail-0013/FP16/person-detection-retail-0013.bin
... 100%, 1411 KB, 30383 KB/s, 0 seconds passed

========== Downloading models/intel/person-detection-retail-0013/INT8/person-detection-retail-0013.xml
... 100%, 1950 KB, 9906 KB/s, 0 seconds passed

========== Downloading models/intel/person-detection-retail-0013/INT8/person-detection-retail-0013.bin
... 100%, 2823 KB, 29574 KB/s, 0 seconds passed

################|| Post-processing

In [4]:
! python3 /opt/intel/openvino/deployment_tools/model_optimizer/mo_caffe.py --input_model models/public/mobilenet-ssd/mobilenet-ssd.caffemodel -o models/mobilenet-ssd/FP32/ --scale 256 --mean_values [127,127,127]

Model Optimizer arguments:
Common parameters:
	- Path to the Input Model: 	/home/u33131/13Nov-SVW-R3/advanced-video-analytics/devcloud/python/models/public/mobilenet-ssd/mobilenet-ssd.caffemodel
	- Path for generated IR: 	/home/u33131/13Nov-SVW-R3/advanced-video-analytics/devcloud/python/models/mobilenet-ssd/FP32/
	- IR output name: 	mobilenet-ssd
	- Log level: 	ERROR
	- Batch: 	Not specified, inherited from the model
	- Input layers: 	Not specified, inherited from the model
	- Output layers: 	Not specified, inherited from the model
	- Input shapes: 	Not specified, inherited from the model
	- Mean values: 	[127,127,127]
	- Scale values: 	Not specified
	- Scale factor: 	256.0
	- Precision of IR: 	FP32
	- Enable fusing: 	True
	- Enable grouped convolutions fusing: 	True
	- Move mean values to preprocess section: 	False
	- Reverse input channels: 	False
Caffe specific parameters:
	- Path to Python Caffe* parser generated from caffe.proto: 	/opt/intel/openvino/deployment_tools/model_optimi

## Part 1. Chaining models: Use mutiple models in an application

The Intel® Distribution of OpenVINO™ toolkit package includes security barrier sample which uses 3 models to detect cars, their number plates, color and number plate attributes from the input video or image of the cars. The sample demo script is provided in the Intel® Distribution of OpenVINO™ toolkit package to run the sample.


    
### 2. Run the executable for the security barrier sample with the mobilenet-ssd* model used in the first tutorial

We will be using the below image in this example to detect multiple attributes from the input image.  

In [ ]:
Image(filename='car_1.png')

Now run the cell below to get the attributes from the image.

In [ ]:
! python3 security_barrier_camera.py -i car_1.png -m models/mobilenet-ssd/FP32/mobilenet-ssd.xml   -l /opt/intel/openvino/deployment_tools/inference_engine/lib/intel64/libcpu_extension_avx2.so -d CPU

### 2.1.See the Output

In [ ]:
Image(filename='security.png')


### 3. Run the security camera sample with Intel optimized pre-trained models


Above script below run the security barrier camera example with Intel® pretrained models. In the script flags, you can see that the sample uses three pretrained models, vehicle-license-plate-detection-barrier, vehicle-attributes-recognition-barrier and license-plate-recognition-barrier to detect the car, it's make, color and license plate attributes. These pretrained models are optimized for particular tasks which yield better performance over generic object detection models. You can find more of such pretrained models under /opt/intel/openvino/deployment_tools/intel_models.

In [ ]:
! python3 security_barrier_camera.py -i car_1.png  -m models/intel/vehicle-license-plate-detection-barrier-0106/FP32/vehicle-license-plate-detection-barrier-0106.xml -m_va models/intel/vehicle-attributes-recognition-barrier-0039/FP32/vehicle-attributes-recognition-barrier-0039.xml -m_lpr models/intel/license-plate-recognition-barrier-0001/FP32/license-plate-recognition-barrier-0001.xml -l /opt/intel/openvino/deployment_tools/inference_engine/lib/intel64/libcpu_extension_avx2.so -d CPU

Following car image will appear the at end of the above command execution. It shows the detection of the car, number plate, its attributes and color. You can view the file by running the cell below

In [ ]:
Image(filename='security.png')






## Part 2. Use multiple models on different hardware

### 0. Initialize the environmental variables

In [ ]:
!/opt/intel/openvino/bin/setupvars.sh


### 1. Let's look at the face detection sample from the Intel® Distribution of OpenVINO™ toolkit package

In [ ]:
!python3 interactive_face_detection.py -h

### 2. Set path to the Input Video

In [ ]:
os.environ["VIDEO"] = "head-pose-female-male.mp4"

We will be using the below video in this example to detect multiple features from the input video.  

In [ ]:
videoHTML('Sample Video', 
          ['head-pose-female-male.mp4'])



### 3. Run the face demo, face detection only, on the Intel® Movidius™ Neural Compute stick


#### Create Job Script 

We will run the workload on several DevCloud's edge compute nodes. We will send work to the edge compute nodes by submitting jobs into a queue. For each job, we will specify the type of the edge compute server that must be allocated for the job.

To pass the specific variables to the Python code, we will use following arguments:

* `-1`&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Location of the output file 
* `-2`&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Location of the input video

The job file will be executed directly on the edge compute node.

In [ ]:
%%writefile infer_face.sh
cd $PBS_O_WORKDIR
mkdir -p $1
python3 interactive_face_detection.py   -m models/intel/face-detection-adas-0001/FP16/face-detection-adas-0001.xml \
                            -i $2 \
                            -o $1 \
                            -d MYRIAD \
                            -l /opt/intel/openvino/deployment_tools/inference_engine/lib/intel64/libcpu_extension_avx2.so 

In [ ]:
#Submit job to the queue
job_id_face = !qsub infer_face.sh -l nodes=1:idc004nc2:intel-ncs2 -F "results/ncs2_face/ $VIDEO" -N obj_det_face
print(job_id_face[0]) 
#Progress indicators
if job_id_face:
    progressIndicator('results/ncs2_face', 'i_progress.txt', "Inferencing", 0, 100)


In [ ]:
videoHTML('Face Detection [MYRIAD] :', 
          ['results/ncs2_face/face.mp4'])


    
### 4. Now we add (to the face detection) also an age and gender detection, running on the CPU

In [ ]:
%%writefile infer_ag.sh
cd $PBS_O_WORKDIR
mkdir -p $1
python3 interactive_face_detection.py   -m models/intel/face-detection-adas-0001/FP16/face-detection-adas-0001.xml \
                            -i ${2} \
                            -o ${1} \
                            -d MYRIAD \
                            -d_ag CPU -m_ag models/intel/age-gender-recognition-retail-0013/FP32/age-gender-recognition-retail-0013.xml \
                            -l /opt/intel/openvino/deployment_tools/inference_engine/lib/intel64/libcpu_extension_avx2.so

In [ ]:
#Submit job to the queue
job_id_ag = !qsub infer_ag.sh -l nodes=1:idc004nc2:intel-ncs2 -F "results/ncs2_ag/ $VIDEO" -N obj_det_ag
print(job_id_ag[0]) 
#Progress indicators
if job_id_ag:
    progressIndicator('results/ncs2_ag', 'i_progress.txt', "Inferencing", 0, 100)

In [ ]:
videoHTML('Face Detection [MYRIAD], Age/Gender [GPU] :', 
          ['results/ncs2_ag/face.mp4'])



### 5. Now we’ll add an head position detection, running on the GPU

In [ ]:
%%writefile infer_hp.sh

#The default path for the job is your home directory, so we change directory to where the files are.
cd $PBS_O_WORKDIR
mkdir -p $1

python3 interactive_face_detection.py   -m models/intel/face-detection-adas-0001/FP16/face-detection-adas-0001.xml\
                            -i ${2} \
                            -o ${1} \
                            -d MYRIAD \
                            -d_ag CPU -m_ag models/intel/age-gender-recognition-retail-0013/FP32/age-gender-recognition-retail-0013.xml \
                            -d_hp GPU -m_hp models/intel/head-pose-estimation-adas-0001/FP16/head-pose-estimation-adas-0001.xml \
                            -l  /opt/intel/openvino/deployment_tools/inference_engine/lib/intel64/libcpu_extension_avx2.so

In [ ]:
#Submit job to the queue
job_id_hp = !qsub infer_hp.sh -l nodes=1:idc004nc2:intel-ncs2 -F "results/ncs2_hp/ $VIDEO" -N obj_det_hp
print(job_id_hp[0]) 
#Progress indicators
if job_id_hp:
    progressIndicator('results/ncs2_hp', 'i_progress.txt', "Inferencing", 0, 100)

In [ ]:
videoHTML('Face Detection[MYRIAD] | Age/Gender[CPU] | Head Pose [GPU] :', 
          ['results/ncs2_hp/face.mp4' ]
          )



### 6. Now we’ll add an emotion detector, running on the GPU

In [ ]:
%%writefile infer_ed.sh
#The default path for the job is your home directory, so we change directory to where the files are.
cd $PBS_O_WORKDIR
mkdir -p $1
python3 interactive_face_detection.py   -m models/intel/face-detection-adas-0001/FP16/face-detection-adas-0001.xml\
                            -i ${2} \
                            -o ${1} \
                            -d MYRIAD \
                            -d_ag CPU -m_ag  models/intel/age-gender-recognition-retail-0013/FP32/age-gender-recognition-retail-0013.xml \
                            -d_hp GPU -m_hp models/intel/head-pose-estimation-adas-0001/FP16/head-pose-estimation-adas-0001.xml\
                            -d_em GPU -m_em models/intel/emotions-recognition-retail-0003/FP16/emotions-recognition-retail-0003.xml \
                            -l /opt/intel/openvino/deployment_tools/inference_engine/lib/intel64/libcpu_extension_avx2.so

In [ ]:
#Submit job to the queue
job_id_ed = !qsub infer_ed.sh -l nodes=1:idc004nc2:intel-ncs2 -F "results/ncs2_ed/ $VIDEO" -N obj_det_ed
print(job_id_ed[0]) 
#Progress indicators
if job_id_ed:
    progressIndicator('results/ncs2_ed', 'i_progress.txt', "Inferencing", 0, 100)

In [ ]:
videoHTML('Face Detection[MYRIAD] | Age/Gender[CPU] | Head Pose, Emotion [GPU] ', 
          ['results/ncs2_ed/face.mp4' ]
          )



### 7. Now let's add facial landmarks detector, running on the CPU

In [ ]:
%%writefile infer_lm.sh

#The default path for the job is your home directory, so we change directory to where the files are.
cd $PBS_O_WORKDIR
mkdir -p $1
python3 interactive_face_detection.py   -m models/intel/face-detection-adas-0001/FP16/face-detection-adas-0001.xml\
                            -i ${2} \
                            -o ${1} \
                            -d MYRIAD \
                            -d_ag CPU -m_ag models/intel/age-gender-recognition-retail-0013/FP32/age-gender-recognition-retail-0013.xml \
                            -d_hp GPU -m_hp models/intel/head-pose-estimation-adas-0001/FP16/head-pose-estimation-adas-0001.xml \
                            -d_em GPU -m_em models/intel/emotions-recognition-retail-0003/FP16/emotions-recognition-retail-0003.xml \
                            -d_lm CPU -m_lm models/intel/facial-landmarks-35-adas-0002/FP16/facial-landmarks-35-adas-0002.xml \
                            -l /opt/intel/openvino/deployment_tools/inference_engine/lib/intel64/libcpu_extension_avx2.so

In [ ]:
#Submit job to the queue
job_id_lm = !qsub infer_lm.sh -l nodes=1:idc004nc2:intel-ncs2 -F "results/ncs2_lm/ $VIDEO" -N obj_det_lm
print(job_id_lm[0]) 
#Progress indicators
if job_id_lm:
    progressIndicator('results/ncs2_lm', 'i_progress.txt', "Inferencing", 0, 100)


In [ ]:
videoHTML('Face Detection[MYRIAD] | Head Pose, Emotion [GPU] | Age/Gender, Landmarks [CPU]', 
          ['results/ncs2_lm/face.mp4' ]
          )